In [0]:
from pyspark.sql import SparkSession


In [0]:
spark = SparkSession.builder.appName("Integrate and manage Public Transport data").getOrCreate()


In [0]:
    spark.conf.set(
        f"fs.azure.account.key.omardbstorageaccount.dfs.core.windows.net", 
        "f5k2suTwYCzz5Pj55WliSmjaKqyZDZ5jy88logUNbHEFNZT60n30rS/hDUIKyUdyQYdm5xMj+Kh/+AStKL6jKQ=="
    )

In [0]:
file_location = "abfss://publictransportdata@omardbstorageaccount.dfs.core.windows.net/raw/"

In [0]:
df = spark.read.format("csv").option("inferSchema", "True").option("header",
"True").option("delimeter",",").load(file_location)

display(df)

In [0]:
from pyspark.sql.functions import date_format, length, when, col


arrival_condition = (length(col("ArrivalTime")) == 8)


df = df.withColumn("ArrivalTime", when(arrival_condition, date_format(df["ArrivalTime"], "HH:mm")).otherwise(df["ArrivalTime"]))



# Format the DepartureTime column
df = df.withColumn("DepartureTime", date_format(df["DepartureTime"], "HH:mm"))



In [0]:
display(df)

In [0]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek

df = df.withColumn("Year", year(df["Date"]))
df = df.withColumn("Month", month(df["Date"]))
df = df.withColumn("Day", dayofmonth(df["Date"]))
df = df.withColumn("DayOfWeek", dayofweek(df["Date"]))


In [0]:
from pyspark.sql.functions import unix_timestamp, col

# Convert DepartureTime and ArrivalTime columns to timestamp
df = df.withColumn("DepartureTimeTimestamp", unix_timestamp(df["DepartureTime"], "HH:mm").cast("timestamp"))
df = df.withColumn("ArrivalTimeTimestamp", unix_timestamp(df["ArrivalTime"], "HH:mm").cast("timestamp"))

# Calculate TripDuration in minutes and hours
df = df.withColumn("TripDurationMinutes", (col("ArrivalTimeTimestamp").cast("long") - col("DepartureTimeTimestamp").cast("long")) / 60) # Duration in minutes


# Drop the intermediate timestamp columns
df = df.drop("DepartureTimeTimestamp", "ArrivalTimeTimestamp")


In [0]:
from pyspark.sql.functions import when

df = df.withColumn("DelayCategory", 
    when(col("Delay") == 0, "Pas de Retard")
    .when((col("Delay") >= 1) & (col("Delay") <= 10), "Retard Court")
    .when((col("Delay") >= 11) & (col("Delay") <= 20), "Retard Moyen")
    .when(col("Delay") > 20, "Long Retard")
    .otherwise("Unknown"))


In [0]:
from pyspark.sql.functions import hour

# Assuming a threshold of 50 passengers for peak hours
threshold = 50

df = df.withColumn("HeureDePointe", when(col("Passengers") >= threshold, "Peak").otherwise("Off-Peak"))


In [0]:
from pyspark.sql import functions as F

route_analysis = df.groupBy("Route").agg(
    F.avg("Delay").alias("AverageDelay"),
    F.avg("Passengers").alias("AveragePassengers"),
    F.count("*").alias("TotalTrips")
)


In [0]:
route_analysis.show()

In [0]:
# Join route-level statistics with the original DataFrame on the "Route" column
df = df.join(route_analysis, on="Route", how="left")


In [0]:
# Define the file location for export
#file_location = "abfss://publictransportdata@omardbstorageaccount.dfs.core.windows.net/processed/"

# Export the DataFrame to the specified location in CSV format
#df.write.csv(file_location, header=True, mode="overwrite")

display(df)